In [1]:
import nltk
import pandas as pd
import csv
from bs4 import BeautifulSoup
import requests
import csv
import numpy as np
from nltk.corpus import stopwords

In [3]:
def getDataSetURL(file_name):
    url = []
    i = 0
    with open(file_name) as filecsv:
        reader = csv.reader(filecsv, delimiter=',')
        for row in reader:
            if i != 0:
                url.append(str(row[0]))
            i += 1

    return url

In [30]:
page_link = getDataSetURL('DataSetArtikel.csv')
article = []
for data in page_link:
    page_response = requests.get(data, timeout=5)
    page_content = BeautifulSoup(page_response.content, "html.parser")
    findP = page_content.find_all("p")

    textContent = []
    for i in range(0, len(findP)):
        paragraphs = findP[i].text
        textContent.append(paragraphs)

    a = []
    i = 0
    findAt = 0
    for data in textContent:
        if 'Klikdokter.com' in data:
            findAt = i
        i += 1

    for i in range(findAt, len(textContent)):
        a.append(textContent[i])

    article.append(a)
np.savetxt('article.txt', article, fmt="%s")

In [88]:
data = []
file = open('article.txt')
spamreader = file.readlines()
for row in spamreader:
    data.append(row.lower()) #mengecilkan huruf kapital
print(data)


['[\'klikdokter.com, jakarta menjelang tahun baru imlek, sejumlah pusat perbelanjaan, restoran, atau rumah-rumah masyarakat tionghoa dipenuhi dengan dekorasi serba merah. warna merah melambangkan semangat, keberanian, keberuntungan, dan kesejahteraan. lalu, adakah pengaruh melihat warna merah terhadap tubuh dan otak manusia?\', \'sejarah warna merah yang identik dengan imlek ini berawal dari sebuah legenda cina tentang nian, seekor binatang buas yang meneror penduduk desa pada tahun baru dan suka memangsa hasil perkebunan, ternak, bahkan anak-anak. penduduk desa percaya bahwa nian takut akan warna merah. sejak itulah warna merah tak bisa dipisahkan dengan imlek.\', \'baca juga\', \'di balik itu, ternyata warna merah bisa memberikan pengaruh tertentu terhadap tubuh dan otak manusia. apa saja efeknya?\', \'sebuah penelitian yang dimuat di jurnal appetite pada tahun 2012 menyebutkan, partisipan yang diberikan makanan dengan piring merah cenderung makan lebih sedikit dibandingkan partisipa

In [143]:
import re
from nltk.tokenize import RegexpTokenizer

#tokenisasi
training_set = data[0:40]
training_set_temp = []
HEADER = 'klikdokter.com'

data_tokenisasi, hasil_tokenisasi = [], []
# tanda_baca = ['"','?','rs/',',','--','.','(',')',"'",'``',"''",'-','[',']','©','%','\\xa0','“','rn/','\\xa0sinus\\xa0dan','\\xa0tim','\\xa0social','nb/','rvs'] #untuk menghilangkan tanda baca

# Hapus header paragraph
for i in training_set:
    temp = i.replace(HEADER, ' ')
    training_set_temp.append(temp)

tokenizer = RegexpTokenizer(r'\w+')
for i in training_set_temp:
    data_tokenisasi.append(tokenizer.tokenize(i))

In [144]:
#membuat unigram menggunakan dictionary
#membuat frekuensi 

kamus = {}
jum_kata_kamus = 0

for j in data_tokenisasi: #menghitung kata-kata yang ada berdasarkan corpus dengan menghitung jumlah keseluruhan kata
    for k in j:
        if (k in kamus): #menghitung jumlah kata yang sudah di dalam kamus
            kamus[k] +=1
        else:
            kamus[k] = 1 #menghitung jumlah kata yang belum di dalam kamus
            jum_kata_kamus += 1 
        # mengetahui jumlah seluruh kata
        
print(kamus)
print(jum_kata_kamus)

{'jakarta': 43, 'menjelang': 1, 'tahun': 65, 'baru': 23, 'imlek': 5, 'sejumlah': 16, 'pusat': 2, 'perbelanjaan': 1, 'restoran': 2, 'atau': 172, 'rumah': 26, 'masyarakat': 21, 'tionghoa': 1, 'dipenuhi': 3, 'dengan': 242, 'dekorasi': 3, 'serba': 2, 'merah': 36, 'warna': 32, 'melambangkan': 1, 'semangat': 3, 'keberanian': 1, 'keberuntungan': 1, 'dan': 494, 'kesejahteraan': 3, 'lalu': 10, 'adakah': 3, 'pengaruh': 3, 'melihat': 8, 'terhadap': 38, 'tubuh': 163, 'otak': 20, 'manusia': 9, 'sejarah': 1, 'yang': 742, 'identik': 2, 'ini': 222, 'berawal': 2, 'dari': 196, 'sebuah': 26, 'legenda': 1, 'cina': 4, 'tentang': 3, 'nian': 2, 'seekor': 1, 'binatang': 2, 'buas': 1, 'meneror': 1, 'penduduk': 5, 'desa': 2, 'pada': 157, 'suka': 2, 'memangsa': 1, 'hasil': 10, 'perkebunan': 2, 'ternak': 1, 'bahkan': 37, 'anak': 30, 'percaya': 8, 'bahwa': 86, 'takut': 3, 'akan': 122, 'sejak': 12, 'itulah': 3, 'tak': 90, 'bisa': 187, 'dipisahkan': 1, 'baca': 42, 'juga': 185, 'di': 268, 'balik': 3, 'itu': 77, 'tern

In [145]:
#baru mencari probabilitas unigram 
#memakai dictionary agar membaca lebih gampang berdasarkan key dan value
probabilitas_unigram = {} 

for x in kamus.keys():
    #menghitung jumlah kata yang dihitung dengan jumlah kata keseluruhan 
    probabilitas_unigram[x] = kamus[x]/jum_kata_kamus
print (probabilitas_unigram)

{'jakarta': 0.01171981466339602, 'menjelang': 0.0002725538293813028, 'tahun': 0.017715998909784682, 'baru': 0.006268738075769964, 'imlek': 0.001362769146906514, 'sejumlah': 0.004360861270100845, 'pusat': 0.0005451076587626056, 'perbelanjaan': 0.0002725538293813028, 'restoran': 0.0005451076587626056, 'atau': 0.04687925865358408, 'rumah': 0.007086399563913873, 'masyarakat': 0.005723630417007359, 'tionghoa': 0.0002725538293813028, 'dipenuhi': 0.0008176614881439084, 'dengan': 0.06595802671027529, 'dekorasi': 0.0008176614881439084, 'serba': 0.0005451076587626056, 'merah': 0.009811937857726901, 'warna': 0.00872172254020169, 'melambangkan': 0.0002725538293813028, 'semangat': 0.0008176614881439084, 'keberanian': 0.0002725538293813028, 'keberuntungan': 0.0002725538293813028, 'dan': 0.13464159171436357, 'kesejahteraan': 0.0008176614881439084, 'lalu': 0.002725538293813028, 'adakah': 0.0008176614881439084, 'pengaruh': 0.0008176614881439084, 'melihat': 0.0021804306350504225, 'terhadap': 0.010357045

In [146]:
#menghitung bigram 
kamus_bigram = {}
jum_kata_kamus_bigram = 0

for j in (data_tokenisasi): #menghitung kata-kata yang ada berdasarkan corpus dengan menghitung jumlah keseluruhan kata
    for k in range(len(j)-1):
        if ((j[k],j[k+1]) in kamus_bigram): #menghitung jumlah kata yang sudah di dalam kamus
            kamus_bigram[(j[k], j[k+1])] +=1
        else:
            kamus_bigram[(j[k], j[k+1])] = 1 #menghitung jumlah kata yang belum di dalam kamus
            # mengetahui jumlah seluruh kata
            jum_kata_kamus_bigram += 1 
        
print((kamus_bigram))
print(jum_kata_kamus_bigram)

{('jakarta', 'menjelang'): 1, ('menjelang', 'tahun'): 1, ('tahun', 'baru'): 10, ('baru', 'imlek'): 1, ('imlek', 'sejumlah'): 1, ('sejumlah', 'pusat'): 1, ('pusat', 'perbelanjaan'): 1, ('perbelanjaan', 'restoran'): 1, ('restoran', 'atau'): 1, ('atau', 'rumah'): 1, ('rumah', 'rumah'): 1, ('rumah', 'masyarakat'): 1, ('masyarakat', 'tionghoa'): 1, ('tionghoa', 'dipenuhi'): 1, ('dipenuhi', 'dengan'): 1, ('dengan', 'dekorasi'): 1, ('dekorasi', 'serba'): 1, ('serba', 'merah'): 2, ('merah', 'warna'): 1, ('warna', 'merah'): 19, ('merah', 'melambangkan'): 1, ('melambangkan', 'semangat'): 1, ('semangat', 'keberanian'): 1, ('keberanian', 'keberuntungan'): 1, ('keberuntungan', 'dan'): 1, ('dan', 'kesejahteraan'): 3, ('kesejahteraan', 'lalu'): 1, ('lalu', 'adakah'): 1, ('adakah', 'pengaruh'): 1, ('pengaruh', 'melihat'): 1, ('melihat', 'warna'): 3, ('merah', 'terhadap'): 1, ('terhadap', 'tubuh'): 3, ('tubuh', 'dan'): 7, ('dan', 'otak'): 3, ('otak', 'manusia'): 3, ('manusia', 'sejarah'): 1, ('sejarah'

In [147]:
print((kamus_bigram[('yang', 'jakarta')]))

KeyError: ('yang', 'jakarta')

In [148]:
#menghitung probabilitas bigram 

#jumlah dari key (0,0+1)/jumlah(key(0))

probabilitas_bigram = {} 

for x in kamus_bigram.keys():
    #menghitung jumlah kata yang dihitung dengan jumlah kata keseluruhan 
    kata0 = x[0]
    kata1 = x[1]
    probabilitas_bigram[(kata0,kata1)] = kamus_bigram[(kata0,kata1)]/kamus[kata0]
print (probabilitas_bigram)

{('jakarta', 'menjelang'): 0.023255813953488372, ('menjelang', 'tahun'): 1.0, ('tahun', 'baru'): 0.15384615384615385, ('baru', 'imlek'): 0.043478260869565216, ('imlek', 'sejumlah'): 0.2, ('sejumlah', 'pusat'): 0.0625, ('pusat', 'perbelanjaan'): 0.5, ('perbelanjaan', 'restoran'): 1.0, ('restoran', 'atau'): 0.5, ('atau', 'rumah'): 0.005813953488372093, ('rumah', 'rumah'): 0.038461538461538464, ('rumah', 'masyarakat'): 0.038461538461538464, ('masyarakat', 'tionghoa'): 0.047619047619047616, ('tionghoa', 'dipenuhi'): 1.0, ('dipenuhi', 'dengan'): 0.3333333333333333, ('dengan', 'dekorasi'): 0.004132231404958678, ('dekorasi', 'serba'): 0.3333333333333333, ('serba', 'merah'): 1.0, ('merah', 'warna'): 0.027777777777777776, ('warna', 'merah'): 0.59375, ('merah', 'melambangkan'): 0.027777777777777776, ('melambangkan', 'semangat'): 1.0, ('semangat', 'keberanian'): 0.3333333333333333, ('keberanian', 'keberuntungan'): 1.0, ('keberuntungan', 'dan'): 1.0, ('dan', 'kesejahteraan'): 0.006072874493927126,

In [149]:
import math

def get_perplexity(prob_bigram):
    pp = 1
    for pair in prob_bigram.keys():
        # Menghindari underflow menggunakan log
        pp += math.log(prob_bigram[pair])
    pp = pp ** (1/len(prob_bigram))
    return pp

In [150]:
# Memprediksi test set
test_set = data[11:15]
arr_tokenisasi_test, tokenisasi_test = [], []

for i in test_set:
    arr_tokenisasi_test.append(nltk.word_tokenize(i))
    
for i in arr_tokenisasi_test:
    for word in i:
        if word not in tanda_baca:
            tokenisasi_test.append(word)
            
# print(probabilitas_bigram[(minta_inputan[0], minta_inputan[1])])
print(len(tokenisasi_test))

2119


In [151]:
# memprediksi kata yang muncul setelah kalimat yang diinputkan
kamus_temp = kamus
for kata in tokenisasi_test:
    if kata not in kamus:
        kamus_temp[kata]= 1

prob_unigram_temp = {}
print(len(kamus_temp))
for key in kamus_temp.keys():
    prob_unigram_temp[key] = kamus_temp[key]/len(kamus_temp)

3745


In [ ]:
# Laplace-smoothed bigram counts

kamus_bigram_temp = {}
keys = list(kamus_temp.keys())

for i in range(len(keys)):
    for j in range(len(keys)):
        if((keys[i], keys[j]) in kamus_bigram):
            kamus_bigram_temp[(keys[i], keys[j])] = kamus_bigram[(keys[i], keys[j])] + 1
        else:
            kamus_bigram_temp[(keys[i], keys[j])] = 1

In [ ]:
print(len(kamus_bigram_temp))

In [ ]:
# Probabilitas bigram setelah di smoothing

prob_bigram_temp = {} 

for x in kamus_bigram_temp.keys():
    #menghitung jumlah kata yang dihitung dengan jumlah kata keseluruhan 
    kata0 = x[0]
    kata1 = x[1]
    prob_bigram_temp[(kata0,kata1)] = kamus_bigram_temp[(kata0,kata1)]/(kamus_temp[kata0] + len(kamus_temp))

In [ ]:
import operator
sorted(prob_bigram_temp.items(), key=operator.itemgetter(1), reverse=True)

In [139]:
print(get_perplexity(prob_bigram_temp))

(1.0000029561019148+5.259552001592032e-07j)


In [57]:
# #memprediksi kata yang muncul setelah 2 kata yang diinputkan
# array_kata2 = []
# nilai_prob_kata2 = []

# for y in probabilitas_bigram.keys(): #melakukan looping untuk melihat keys yang ada di porbabilitas bigram 
#     #melakukan pengecekan 
#     if (minta_inputan[1] == y[0]): #ketika sama akan dimasukkan ke array baru
#         array_kata2.append(y[1])
#         nilai_prob_kata2.append(probabilitas_bigram[(minta_inputan[1], y[1])])
# #     else:
        
# maksimum = nilai_prob_kata2[0]
# for looping2 in range(len(nilai_prob_kata2)): #bisa menyamakan indeksnya
#     if (maksimum < nilai_prob_kata2[looping2]):
#         maksimum = nilai_prob_kata2[looping2]
#         kata_tetap = array_kata2[looping2]
    
# print(maksimum) #mengeluarkan probabilitas terbaik atau maksimum dari kata2 yang telah dicari didalam bigram
# print(kata_tetap) #kata teteap merupakan mengeluarkan kata yang dikeluarkan setelah kata yang diinputkan
# print(array_kata2) #mengeluarkan kata-kata selanjutnya yang berhubungan dengan per-kata yang diinputkan 
        